In [1]:
%load_ext autotime
%load_ext autoreload
%autoreload 2
%matplotlib inline 
#inline
# %pdb
%config InlineBackend.figure_format ='retina'

import os
import pathlib
import sys

## setting path
## set the base path
# os.chdir(pathlib.Path(os.path.dirname(path[0])).parent.absolute())
# ## set path to sys for Lmbda functio
# sys.path.insert(0, os.getcwd())

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

time: 511 ms (started: 2023-03-27 15:37:19 +02:00)


## IMPORT

In [2]:
import feast
from feast import FeatureStore

import sqlalchemy
import yaml
from typing import Dict, Union
from copy import deepcopy

import pandas as pd
from feast.repo_config import RegistryConfig, RepoConfig
from feast.infra.online_stores.redis import RedisOnlineStoreConfig
from feast.infra.offline_stores.contrib.postgres_offline_store.postgres import PostgreSQLOfflineStoreConfig

time: 1.2 s (started: 2023-03-27 15:37:20 +02:00)


## ENV Variables

In [3]:
repoPath = "/home/sourabh/Resonanz/Algotrader/zenml/ireland-old/ipynb/featurestore/env/staging"

time: 14.4 ms (started: 2023-03-27 15:37:21 +02:00)


## LOAD

In [7]:
dd = [{'epoch_time' : int(pd.Timestamp(d.strftime('%Y-%m-%d %H:%M:%S')).timestamp())} for d in pd.date_range(start='2019-03-17', end='2023-03-29', freq='H')]

repoConfig = RepoConfig(
        registry=RegistryConfig(path="s3://bayaga/ireland/registry.pb"),
        project="IDA",
        provider="local",
#         offline_store = PostgreSQLOfflineStoreConfig(
#               host= "aws-ec2-db-prod-1adbe0f5",
#               port= "5432",
#               database= "algotrader",
#               password= "1234abc",
#               db_schema= "public",
#               user= "usr__sourabh_raj",
#               sslmode= "verify-full",
#               sslkey_path= "/home/sourabh/Resonanz/Algotrader/env/sourabh/prod/usr__sourabh_raj.key",
#               sslcert_path= "/home/sourabh/Resonanz/Algotrader/env/sourabh/prod/usr__sourabh_raj.crt",
#               sslrootcert_path= "/home/sourabh/Resonanz/Algotrader/env/sourabh/prod/ca.crt",
#         ),
        online_store=RedisOnlineStoreConfig(connection_string="localhost:6379"),
        entity_key_serialization_version=2,
    )
store = FeatureStore(config=repoConfig)
# store = FeatureStore(repo_path=repoPath)

batch_scoring_features = store.get_online_features(
    entity_rows=dd,
#     features = ['feature_lag_fund:day_avg_lag2', 
#                 'feature_lag_fund:day', 'feature_lag_fund:lag30', 'feature_lag_fund:rolling_2_avg_lag2']
    features=store.get_feature_service("ireland_data_service", allow_cache=True),
).to_df()
# rolling_7_avg_lag2	day_avg_lag30	rolling_30_median_lag2	day_median_lag7	month	lag4

batch_scoring_features.set_index('epoch_time', inplace=True)
batch_scoring_features.index = pd.to_datetime(batch_scoring_features.index, unit='s')
batch_scoring_features.index.name = None

batch_scoring_features

,price_diff,ie_actual_wnd,lag4,day_median_lag30,rolling_3_avg_lag2,year_day,quarter,day_median_lag2,day_avg_lag2,lag5,...,week,rolling_7_median_lag2,rolling_2_avg_lag2,rolling_30_median_lag2,lag6,da_price,demand_enappsys,wind_ecop6,wind_change_ecop6,enappsys_ecop6_residual
2019-03-17 00:00:00,27.950,1445.25,28.720,0.34,-32.798333,76.0,1.0,-28.7625,-47.07,-24.570,...,11.0,16.9850,-63.5575,-6.180,130.550,44.62,4220.5,2324.3075,NaN,1896.1925
2019-03-17 01:00:00,80.580,1260.50,32.925,0.34,-24.196667,76.0,1.0,-28.7625,-47.07,9.225,...,11.0,13.4450,-52.7575,2.615,86.645,43.53,3949.0,2185.3850,-138.9225,1763.6150
2019-03-17 02:00:00,49.085,1275.50,15.620,0.34,-42.916667,76.0,1.0,-28.7625,-47.07,-33.345,...,11.0,2.7275,-72.1850,0.000,25.280,43.53,3716.5,2046.1975,-139.1875,1670.3025
2019-03-17 03:00:00,-31.325,1258.00,30.840,0.34,-6.830000,76.0,1.0,-28.7625,-47.07,-28.130,...,11.0,-4.8200,-25.6650,-1.820,0.805,43.53,3500.5,1869.7300,-176.4675,1630.7700
2019-03-17 04:00:00,-6.300,1224.75,21.630,0.34,8.858333,76.0,1.0,-28.7625,-47.07,-31.870,...,11.0,2.4475,2.4725,-1.135,23.105,40.60,3315.0,1793.0325,-76.6975,1521.9675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-28 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-28 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-28 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-28 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


time: 17.6 s (started: 2023-03-27 15:39:04 +02:00)


In [ ]:
store._registry.list_request_feature_views('IDA')

# df = batch_scoring_features.copy()

# df.set_index('epoch_time', inplace=True)
# df.index = pd.to_datetime(df.index, unit='s')

In [ ]:
# from zenml.client import Client

# Client().active_stack.model_deployer.find_model_servers()